In [ ]:
import pandas as pd

# Load CSV
df = pd.read_csv(r"train.csv")
df.head()

,title,substance,age,weight,gender,report_text,word_count
0,Almost the End,LSD,Not Given,48.0,Female,2 weeks ago my friends and i drove to dallas t...,906
1,Archetypal initiation,DMT,Not Given,77.0,Male,could never hope to capture the essence of the...,1463
2,A Magical Experience,Psilocybin,Not Given,70.0,Male,"1st, the day after halloween, unluckily having...",1435
3,Colorful and Enlightening,Psilocybin,Not Given,70.0,Male,i have been drug free my entire life. i am cur...,1006
4,More Common Ground,Psilocybin,Not Given,NaN,Female,"dose:mushrooms mdmacannabis i have two sons, z...",860


In [2]:
df.shape

(1920, 7)

In [3]:
from symspellpy.symspellpy import SymSpell, Verbosity
import re

In [5]:
# Initialize SymSpell
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

dictionary_path = "frequency_dictionary_en_82_765.txt" 

# Load dictionary
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

In [6]:
df["report_text"] = df["report_text"].str.replace(r"\s+", " ", regex=True).str.strip()

In [7]:
df.head()

,title,substance,age,weight,gender,report_text,word_count
0,Almost the End,LSD,Not Given,48.0,Female,2 weeks ago my friends and i drove to dallas t...,906
1,Archetypal initiation,DMT,Not Given,77.0,Male,could never hope to capture the essence of the...,1463
2,A Magical Experience,Psilocybin,Not Given,70.0,Male,"1st, the day after halloween, unluckily having...",1435
3,Colorful and Enlightening,Psilocybin,Not Given,70.0,Male,i have been drug free my entire life. i am cur...,1006
4,More Common Ground,Psilocybin,Not Given,NaN,Female,"dose:mushrooms mdmacannabis i have two sons, z...",860


In [9]:
from collections import Counter

def get_spelling_errors(text):
    words = re.findall(r'\b[a-zA-Z]+\b', text.lower())  # Extract alphabetic words
    errors = []
    for word in words:
        # If SymSpell finds no correction for the word, treat it as misspelled
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if len(suggestions) == 0:
            errors.append(word)
    return errors

In [11]:
# Apply to all reports
df["spelling_errors"] = df["report_text"].apply(get_spelling_errors)
df["num_errors"] = df["spelling_errors"].apply(len)

# Get all unique spelling errors across the dataset
all_errors = df["spelling_errors"].explode()
error_counts = Counter(all_errors)

# View top 20
pd.DataFrame(error_counts.most_common(50), columns=["misspelled_word", "count"])

,misspelled_word,count
0,NaN,769
1,cubensis,156
2,ayahuasca,104
3,headspace,64
4,tryptamine,47
5,adderall,37
6,psychonaut,37
7,divinorum,32
8,entheogens,31
9,psytrance,31
